In [14]:
%%capture
! pip install boto3

In [2]:
import boto3
import json
import math
import numpy as np
import requests
import re

## Find course description from course name at NYU using Course Catalog API

In [20]:
with open('rootkey.json', 'r') as json_file:
    rootkey = json.load(json_file)

In [21]:
course_name = "Data Structures and Algorithms" # placeholder test course name
course_name = course_name.replace(" ", "+")

url = 'https://sandbox.api.it.nyu.edu/course-catalog-exp/courses?course_title=' + course_name
headers = {'Authorization': rootkey['NYUAccessKey']}

req = requests.get(url, headers=headers)

# print(req.status_code)
# print(req.headers)
# print(req.text)

course_reqs = json.loads(req.text)
allsentences = []
if len(course_reqs) == 0:
    print("Sorry, couldn't find any courses like that!")
else:
    for i in range(len(course_reqs)):
        course_des = course_reqs[i]["course_descr"]
        allsentences.append(course_des)

# print(allsentences)

Sorry, couldn't find any courses like that!


### Generate bag of words and create vector for TF-IDF

In [9]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ericm\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
stop_words = set(stopwords.words('english')) 

def word_cleanup(sentence):
    words = re.sub("[^\w]", " ",  sentence).split()
    cleaned_text = [w.lower() for w in words if w not in stop_words and w.isalpha()]
    return cleaned_text

def tokenize(sentences):
    words = []
    for sentence in sentences:
        w = word_cleanup(sentence)
        words.extend(w)
    words = sorted(list(set(words)))
    return words

def generate_bow(allsentences):    
    vocab = tokenize(allsentences)
    print("Bag of Words for Document \n{0} \n".format(vocab));
    
generate_bow(allsentences)
    
for sentence in allsentences:
    words = word_cleanup(sentence)
    bag_vector = np.zeros(len(vocab))
    for w in words:
        for i, word in enumerate(vocab):
            if word == w: 
                bag_vector[i] += 1

    print("{0}\n{1}\n".format(sentence,np.array(bag_vector)))

['abstract', 'algorithm', 'algorithms', 'allocation', 'along', 'analysis', 'available', 'basics', 'better', 'binary', 'c', 'comparison', 'complexity', 'computational', 'corequisite', 'course', 'covers', 'cs', 'data', 'design', 'dynamic', 'ex', 'fundamental', 'general', 'graduate', 'gy', 'hashing', 'heap', 'implementation', 'include', 'introduces', 'linked', 'mergesort', 'methods', 'not', 'online', 'open', 'prerequisite', 'program', 'queues', 'quicksort', 'searching', 'selection', 'sequential', 'sort', 'sorting', 'specifications', 'stacks', 'standard', 'standing', 'storage', 'structures', 'students', 'taken', 'techniques', 'this', 'topics', 'trees', 'types', 'use', 'uy', 'version']
Word List for Document 
[] 



### AWS Comprehend Detect Key Phrases

In [ ]:
with open('rootkey.json', 'r') as json_file:
    rootkey = json.load(json_file)

In [ ]:
course_des1 = course_reqs[0]["course_descr"]
comprehend = boto3.client(service_name='comprehend', region_name='us-east-1',
                          aws_access_key_id=rootkey['AWSAccessKeyId'],
                          aws_secret_access_key=rootkey['AWSSecretKey'])

print(json.dumps(comprehend.detect_key_phrases(Text=course_des1, LanguageCode='en')))

In [ ]:
keywords = json.loads(json.dumps(comprehend.detect_key_phrases(Text=course_des1, LanguageCode='en')))
for i in range(len(keywords["KeyPhrases"])//2): #only displaying half
    keyword = keywords["KeyPhrases"][i]["Text"]
    print(keyword)

## List of all Organizations/Clubs at NYU using Engage API

In [7]:
url = 'https://sandbox.api.it.nyu.edu/engage-exp/organizations'
headers = {'Authorization': rootkey['NYUAccessKey']}

req = requests.get(url, headers=headers)

# print(req.status_code)
# print(req.headers)
# print(req.text)

org_reqs = json.loads(req.text)
# print(req[0])
print(len(req))
print(org_reqs[0]["description"] + org_reqs[0]["keywords"])
print(org_reqs[0]["long_name"])

955
To help students turn to Christ and live for Christ.180, Fellowship, Christian, Small Groups, Purpose, Christianity
180 Fellowship - All-University


In [ ]:
def termFrequency(term, document):
    normalizeDocument = document.lower().split()
    return normalizeDocument.count(term.lower()) / float(len(normalizeDocument))

def inverseDocumentFrequency(term, allDocuments):
    numDocumentsWithThisTerm = 0
    for doc in allDocuments:
        if term.lower() in allDocuments[doc].lower().split():
            numDocumentsWithThisTerm = numDocumentsWithThisTerm + 1
 
    if numDocumentsWithThisTerm > 0:
        return 1.0 + log(float(len(allDocuments)) / numDocumentsWithThisTerm)
    else:
        return 1.0

#### Cosine Similarity

In [ ]:
def compute_cosine_similarity(vector1, vector2):
  '''
  Computes the cosine similarity of the two input vectors.

  Inputs:
    vector1: A nx1 numpy array
    vector2: A nx1 numpy array

  Returns:
    A scalar similarity value.
  '''
  def norm(vector):
    return math.sqrt(np.dot(vector, vector))

  denominator = (norm(vector1) * norm(vector2))
  return np.dot(vector1, vector2) / denominator if denominator != 0 else 0